In [ ]:
topics = """
politics
socioeconomy
culture
gender
religion
sociology
social class
technology
ethnicity
infrastructure
health
education
ecology
party politics
diplomacy
history
cuisine
microeconomics
business
artificial intelligence
law
empowerment of youths
perdana menteri
hutang negara
kes rasuah
isu kemiskinan
sumber asli
"""
topics = [t.strip() for t in topics.strip().split('\n')]
len(topics)

In [ ]:
import json
import os

unique = set()
filtered = []
with open('generated-roleplay.jsonl') as fopen:
    for l in fopen:
        l = json.loads(l)
        for r in l['roleplay']:
            lower = r['persona_ms'].lower()
            if lower not in unique:
                unique.add(lower)
                filtered.append(r)
len(filtered)

In [ ]:
r

In [ ]:
filtered[0]

In [ ]:
len(filtered) * len(topics)

In [ ]:
prompts = []
for d in filtered:
    n = f"{d['name']}. {d['persona_ms']}. {d['description_ms']}"
    for t in topics:
        s = f"""
user B is {n}

generate chat session in malay related to {t} malaysia, user A user B, DO NOT EXPLAIN, return in JSON [{{'user', 'message'}}]
""".strip()
        prompts.append(s)

In [ ]:
len(prompts)

In [ ]:
d

In [ ]:
print(prompts[0])

In [ ]:
directory = 'generate-roleplay-conversation'
# !rm -rf {directory}
!mkdir {directory}

In [ ]:
from openai import OpenAI
import requests

headers = {
    'accept': 'application/json',
    'Content-Type': 'application/json',
}

def answer(q, i):
    filename = f'{directory}/{i}.json'
    if os.path.exists(filename):
        return
    
    results = []
    openai = OpenAI(
        base_url='https://llama-3.us.mesolitica.com/v1',
        api_key='empty',
    )
    for _ in range(1):
        
        while True:
            try:
                
                json_data = {
                    'messages': [
                        {
                            'role': 'user',
                            'content': q,
                        },
                    ],
                    'model': 'model',
                    'stop': [
                        '<|eot_id|>',
                    ],
                    'temperature': 0.9,
                    'max_tokens': 2048,
                }
                response = requests.post(
                    '', 
                    headers=headers, json=json_data, timeout = 60 * 10)
                
                r = response.json()['choices'][0]['message']['content']
                results.append(r)
                break
            except Exception as e:
                # print(e)
                pass
                
    
    with open(filename, 'w') as fopen:
        json.dump(results, fopen)

In [ ]:
def consumer(queue, name):
    while True:
        if queue.qsize() == 0:
            break
        item = queue.get()
        answer(*item)
    print(f'consumer {name} done')

In [ ]:
urls = [(q, no) for no, q in enumerate(prompts)]

In [ ]:
answer(*urls[0])

In [ ]:
from threading import Thread
from queue import Queue
from tqdm import tqdm

queue = Queue()
for u in urls:
    queue.put(u)
    
ori_size = queue.qsize()

In [ ]:
max_worker = 50
consumers = [Thread(target=consumer, args=(queue,i)) for i in range(max_worker)]
for i in range(len(consumers)):
    consumers[i].start()
    
pbar = tqdm(total=ori_size)
last_size = 0
while True:
    size = queue.qsize()
    if size == 0:
        break
    left = ori_size - size
    minus = left - last_size
    if minus > 0:
        pbar.update(minus)
        last_size += minus

pbar.close()